In [10]:
from PLL import PhaseLockedLoop
import matplotlib.pyplot as plt
import numpy as np

class PhaseDetector:
    def __init__(self) -> None:
        pass

    def proc( self, sigIn, sigOut ):
        return sigIn * sigOut

# Yes, this one is immediate in the original
class LoopFilter:
    def __init__( self, K_i, K_p, fs ) -> None:
        self.K_i = K_i
        self.K_p = K_p
        self.integrator = 0
        self.fs = fs
    
    def proc( self, e_D ):
        self.integrator += self.K_i * e_D / self.fs
        return self.K_p * e_D + self.integrator

    

class NumericallyControlledOscillator:
    def __init__( self, K_0, f, fs ) -> None:
        self.K_0 = K_0
        self.f = f
        self.fs = fs
        self.theta = 0
        self.phaseEstimate = 0

    def proc( self, e_F ):
        self.theta += 2*np.pi*self.f/self.fs + self.K_0 * e_F
        if self.theta > 2*np.pi:
            self.theta -= 2*np.pi
        
        cosOut = np.cos( self.theta  )
        sinOut = -np.sin( self.theta  )
        return cosOut, sinOut
    

class Fast_PhaseLockedLoop:
    def __init__( self, K_i, K_p, K_0, pll_f, fs ) -> None:
        self.pd = PhaseDetector()
        self.lf = LoopFilter( K_i, K_p, fs )
        self.nco = NumericallyControlledOscillator( K_0, pll_f, fs )
        self.lastSinOut = 0
        
        self.lf_integrator = 0
        
    def proc( self, in_, ):
#         e_D = self.pd.proc( in_, self.lastSinOut )
        e_D = in_ * self.lastSinOut

        e_F = self.lf.proc( e_D )
        
        cosOut, sinOut  = self.nco.proc(e_F)
        self.lastSinOut = sinOut
        return cosOut, sinOut, e_D, e_F

K = 1
duration_secs = 2 / K
in_f_Hz   = 10*K
fs = 150 * K * 10

t = np.linspace(0,duration_secs, int(duration_secs*fs))
in_sig    = np.cos( 2*np.pi * in_f_Hz*1.25 * t + 0 )

for K_p in [0.2667/10, 0.2667, 0.1337/10,]:
    for K_i in [0.0178*100, 0.0178, 0.0178*10, 0.1337]:
        for K_0 in [1, 2, 5]:
            for in_f_Hz in [ 9900 ]:
                pll = PhaseLockedLoop( K_i, K_p, K_0, in_f_Hz, fs )
                fast_pll = Fast_PhaseLockedLoop( K_i, K_p, K_0, in_f_Hz, fs )
                
                for idx in range(len(in_sig)):
                    orig_cosOut, orig_sinOut, orig_e_D, orig_e_F = pll.proc( in_sig[idx] )
                    
                    fast_cosOut, fast_sinOut, fast_e_D, fast_e_F = fast_pll.proc( in_sig[idx] )
                    
                    assert( orig_cosOut == fast_cosOut )
                    assert( orig_sinOut == fast_sinOut )
                    assert( orig_e_D == fast_e_D )
                    assert( orig_e_F == fast_e_F )
                    
                    

print( "Done" )

AssertionError: 